In [65]:
#fit ten different models 3 logistic regressors, 3 decision trees, 2 knns
#and 2 rfc the saw how they did in a voting classifier
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier
pd.set_option('display.max_columns', 500)
import pickle as pkl

In [66]:
df = pd.read_csv('../prepeddata.csv',index_col='Unnamed: 0')
df['stimulant'] = df['stimulant'].apply(lambda x:'low' if x <= 2 else 'high')
df


,Age,Education,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SensationSeeking,Alchol,Caff,Chocalate,Nicotine,stimulant,depressant,hallucinagen
1,3,1,39.0,36.0,42.0,37.0,42.0,4.0,3.0,5,6,5,2,low,1.0,0.000000
2,2,4,29.0,52.0,55.0,48.0,41.0,3.0,6.0,5,6,6,4,low,1.0,2.000000
3,3,1,31.0,45.0,40.0,32.0,34.0,2.0,8.0,6,6,4,0,low,0.0,0.333333
4,1,3,34.0,34.0,46.0,47.0,46.0,2.0,3.0,4,5,4,2,low,1.5,0.000000
5,3,4,43.0,28.0,43.0,41.0,50.0,4.0,6.0,4,6,6,2,low,0.5,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,1,0,25.0,51.0,57.0,48.0,33.0,7.0,11.0,5,4,4,0,low,0.0,1.000000
1885,1,0,33.0,51.0,50.0,48.0,30.0,7.0,9.0,5,5,4,5,low,0.0,3.666667
1886,2,2,47.0,30.0,37.0,31.0,31.0,6.0,5.0,4,6,6,6,high,5.0,2.666667
1887,1,0,45.0,26.0,48.0,32.0,22.0,8.0,10.0,5,6,5,4,low,0.0,3.000000


In [36]:
stimx = df.drop(['stimulant','hallucinagen','depressant'],axis = 1)

stimy = df.stimulant
poly = PolynomialFeatures(degree=2,interaction_only=True)
polystimtestdata = poly.fit_transform(stimx)
polystimtestdata = SelectKBest(k=15).fit_transform(polystimtestdata,stimy)



C:\Users\Sigma\AppData\Roaming\Python\Python37\site-packages\sklearn\feature_selection\_univariate_selection.py:115: UserWarning: Features [0] are constant.
  UserWarning)
C:\Users\Sigma\AppData\Roaming\Python\Python37\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [37]:
stimXtrain,stimXtest,stimYtrain,stimYtest = train_test_split(stimx,stimy)
stimXtrain

,Age,Education,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SensationSeeking,Alchol,Caff,Chocalate,Nicotine
1009,4,3,42.0,31.0,42.0,40.0,33.0,4.0,6.0,6,4,6,6
216,2,2,51.0,38.0,41.0,50.0,39.0,5.0,7.0,5,6,4,3
1752,2,2,54.0,44.0,55.0,36.0,37.0,8.0,10.0,5,6,5,2
1101,1,0,36.0,37.0,46.0,37.0,37.0,2.0,6.0,4,6,4,0
1401,2,3,46.0,46.0,39.0,39.0,41.0,2.0,3.0,5,6,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,1,0,43.0,47.0,55.0,33.0,28.0,7.0,9.0,6,5,5,6
273,5,4,26.0,40.0,46.0,45.0,42.0,6.0,4.0,6,6,6,1
152,3,2,41.0,28.0,33.0,46.0,35.0,3.0,4.0,5,6,6,2
1377,3,2,50.0,36.0,52.0,32.0,39.0,4.0,9.0,5,6,6,2


In [38]:
stimscaler = StandardScaler()
stimXtrain = stimscaler.fit_transform(stimXtrain)
stimXtest = stimscaler.transform(stimXtest)
stimXtrain[0]

array([ 1.27503257,  1.14033559,  0.68058764, -1.28484622, -0.57351605,
       -0.45874306, -1.22206598, -0.37015698, -0.21734135,  1.0163558 ,
       -1.28773583,  0.82354524,  1.17449549])

Logistic regression classifiers

In [39]:
mod = LogisticRegression(class_weight = 'balanced'
                         ,max_iter=1000,solver='liblinear',warm_start=True,tol=0.2)
mod.fit(stimXtrain,stimYtrain)
stimtrianlogypred = mod.predict(stimXtrain)
stimtestlogypred = mod.predict(stimXtest)

In [40]:
f1_score(stimYtrain,stimtrianlogypred,average='weighted'),f1_score(stimYtest,stimtestlogypred,average='weighted')

(0.6106317174778456, 0.6069574651702428)

In [41]:
mod2 = LogisticRegression(class_weight='balanced',max_iter=4000,solver='sag',warm_start=True,tol=0.1)
mod2.fit(stimXtrain,stimYtrain)
stimtrianlog2ypred = mod2.predict(stimXtrain)
stimtestlog2ypred = mod2.predict(stimXtest)

In [42]:
f1_score(stimYtrain,stimtrianlog2ypred,average='weighted'),f1_score(stimYtest,stimtestlog2ypred,average='weighted')

(0.5633653007646549, 0.5309734883985867)

In [43]:
mod3 = LogisticRegression(class_weight='balanced',max_iter=4000,solver='liblinear')
mod3.fit(stimXtrain,stimYtrain)
stimtrianlog3ypred = mod3.predict(stimXtrain)
stimtestlog3ypred = mod3.predict(stimXtest)

In [44]:
f1_score(stimYtrain,stimtrianlog3ypred,average='weighted'),f1_score(stimYtest,stimtestlog3ypred,average='weighted')

(0.6098041185559305, 0.6069574651702428)

Decision Trees

In [45]:
dtc = DecisionTreeClassifier(criterion='gini',ccp_alpha=0.0025)
dtc.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc.predict(stimXtrain)
dtcytestpred = dtc.predict(stimXtest)

In [46]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')




(0.654401911373999, 0.5744061997893812)

In [47]:
dtc2 = DecisionTreeClassifier(criterion='entropy',max_depth=50,ccp_alpha=0.0052)
dtc2.fit(stimXtrain,stimYtrain)
dtc2ytrainpred = dtc2.predict(stimXtrain)
dtc2ytestpred = dtc2.predict(stimXtest)

In [48]:
f1_score(stimYtrain,dtc2ytrainpred,average='weighted'),f1_score(stimYtest,dtc2ytestpred,average='weighted')


(0.6448402419871241, 0.5404337377924282)

In [49]:
dtc3 = DecisionTreeClassifier(class_weight='balanced',criterion='gini',ccp_alpha=0.005)
dtc3.fit(stimXtrain,stimYtrain)
dtc3ytrainpred = dtc3.predict(stimXtrain)
dtc3ytestpred = dtc3.predict(stimXtest)

In [50]:
f1_score(stimYtrain,dtc3ytrainpred,average='weighted'),f1_score(stimYtest,dtc3ytestpred,average='weighted')


(0.48288512056149446, 0.46237962720418574)

Knearest

In [51]:
knn = KNeighborsClassifier(n_neighbors=43,weights='uniform')
knn.fit(stimXtrain,stimYtrain)
knntrainypred=knn.predict(stimXtrain)
knntestypred=knn.predict(stimXtest)
f1_score(stimYtrain,knntrainypred,average='weighted'),f1_score(stimYtest,knntestypred,average='weighted')

(0.6001986605538645, 0.5707193045556369)

In [52]:
knn2 = KNeighborsClassifier(n_neighbors=11,weights='uniform')
knn2.fit(stimXtrain,stimYtrain)
knn2trainypred=knn2.predict(stimXtrain)
knn2testypred=knn2.predict(stimXtest)
f1_score(stimYtrain,knn2trainypred,average='weighted'),f1_score(stimYtest,knn2testypred,average='weighted')

(0.6521592517454168, 0.5541867538593109)

Random Forest

In [53]:
rfc = RandomForestClassifier(criterion='gini',ccp_alpha=0.005,n_estimators=1000,max_features=6,class_weight='balanced',max_depth=6,oob_score=True)
rfc.fit(stimXtrain,stimYtrain)

RandomForestClassifier(ccp_alpha=0.005, class_weight='balanced', max_depth=6,
                       max_features=6, n_estimators=1000, oob_score=True)

In [54]:
rfctrainypred = rfc.predict(stimXtrain)
rfctestypred = rfc.predict(stimXtest)
f1_score(rfctrainypred,stimYtrain,average='weighted'),f1_score(rfctestypred,stimYtest,average='weighted')

(0.6078831389938153, 0.5406070759447759)

In [55]:
rfc2 = RandomForestClassifier(criterion='entropy',ccp_alpha=0.01,n_estimators=10000,class_weight='balanced',oob_score=True)
rfc2.fit(stimXtrain,stimYtrain)

RandomForestClassifier(ccp_alpha=0.01, class_weight='balanced',
                       criterion='entropy', n_estimators=10000, oob_score=True)

In [56]:
rfc2trainypred = rfc2.predict(stimXtrain)
rfc2testypred = rfc2.predict(stimXtest)
f1_score(rfc2trainypred,stimYtrain,average='weighted'),f1_score(rfc2testypred,stimYtest,average='weighted')

(0.6393056502283551, 0.563496382419524)

voting classifier

In [62]:
vtc = VotingClassifier([('dtc2',dtc2),('rfc2',rfc2),('knn',knn),('knn2',knn2),('log',mod),('log3',mod3),('log2',mod2),('dtc',dtc),('dtc3',dtc3),("rfc",rfc)],voting='soft',n_jobs=-1)
vtc.fit(stimXtrain,stimYtrain)

VotingClassifier(estimators=[('dtc2',
                              DecisionTreeClassifier(ccp_alpha=0.0052,
                                                     criterion='entropy',
                                                     max_depth=50)),
                             ('rfc2',
                              RandomForestClassifier(ccp_alpha=0.01,
                                                     class_weight='balanced',
                                                     criterion='entropy',
                                                     n_estimators=10000,
                                                     oob_score=True)),
                             ('knn', KNeighborsClassifier(n_neighbors=43)),
                             ('knn2', KNeighborsClassifier(n_neighbors=11)),
                             ('log',
                              LogisticRegres...
                              LogisticRegression(class_weight='balanced',
                                 

In [63]:
vtcypred = vtc.predict(stimXtest)
vtcytrain = vtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))


0.6741256212977288 0.6092795665400677


In [64]:
pkl.dump(vtc,open('vtc.pkl','wb'))